# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

Cities_DF = pd.read_csv('WeatherPy.csv')
Cities_DF

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,kahului,US,78.82,74,0,13.80,20.8947,-156.4700,1660983394
1,hambantota,LK,87.53,62,78,24.05,6.1241,81.1185,1660983395
2,mulege,MX,83.91,62,100,2.95,26.8833,-111.9833,1660983395
3,ponta delgada,PT,66.92,82,40,5.75,37.7333,-25.6667,1660983395
4,amos,CA,63.18,86,80,6.73,48.5669,-78.1162,1660983395
...,...,...,...,...,...,...,...,...,...
545,uruzgan,AF,79.52,36,100,1.72,32.8333,66.0000,1660983488
546,tivoli,IT,77.58,82,0,3.76,41.9609,12.7989,1660983488
547,praia da vitoria,PT,71.76,94,40,2.24,38.7333,-27.0667,1660983488
548,lamar,US,77.36,88,75,5.75,33.6668,-95.5836,1660983489


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#get locations and humidity
city_locations = Cities_DF[['Latitude', 'Longitude']].astype(float)
humidity = Cities_DF['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=150,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_criteria_df = Cities_DF.loc[(Cities_DF['Temperature'] >= 70) & (Cities_DF['Temperature'] <= 80) & 
                                  (Cities_DF['Windspeed'] <= 10) & (Cities_DF['Cloudiness'] == 0)]
weather_criteria_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
15,zonguldak,TR,76.05,50,0,3.47,41.2500,31.8333,1660983399
20,laguna,US,75.94,65,0,4.61,38.4210,-121.4238,1660983400
37,bethel,US,72.63,84,0,0.00,41.3712,-73.4140,1660983222
50,ponta do sol,PT,70.43,70,0,1.57,32.6667,-17.1000,1660983406
98,yarmouth,US,72.97,79,0,6.91,41.7057,-70.2286,1660983414
106,geraldton,AU,72.01,60,0,8.05,-28.7667,114.6000,1660983415
137,palmer,US,70.86,81,0,0.00,42.1584,-72.3287,1660983419
145,nigde,TR,74.93,36,0,6.38,37.8333,34.7500,1660983135
152,sussex,US,71.15,80,0,1.01,41.1834,-74.6663,1660983424
184,carnarvon,AU,78.66,28,0,2.86,-24.8667,113.6333,1660983430


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a hotel_df
hotel_df = weather_criteria_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
15,zonguldak,TR,41.2500,31.8333,
20,laguna,US,38.4210,-121.4238,
37,bethel,US,41.3712,-73.4140,
50,ponta do sol,PT,32.6667,-17.1000,
98,yarmouth,US,41.7057,-70.2286,
106,geraldton,AU,-28.7667,114.6000,
137,palmer,US,42.1584,-72.3287,
145,nigde,TR,37.8333,34.7500,
152,sussex,US,41.1834,-74.6663,
184,carnarvon,AU,-24.8667,113.6333,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [7]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
#     time.sleep(1)

# Print -- once searching is completed
print("--------------")

Retrieving Results for Index 15: zonguldak.
No result... skipping.
------------
Retrieving Results for Index 20: laguna.
Closest hotel in laguna is Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5.
------------
Retrieving Results for Index 37: bethel.
Closest hotel in bethel is Hampton Inn Danbury.
------------
Retrieving Results for Index 50: ponta do sol.
Closest hotel in ponta do sol is 1905 Zinos Palace.
------------
Retrieving Results for Index 98: yarmouth.
Closest hotel in yarmouth is Isaiah Hall Bed & Breakfast.
------------
Retrieving Results for Index 106: geraldton.
Closest hotel in geraldton is Ocean Centre Hotel - Stay Beachside in Geraldton!.
------------
Retrieving Results for Index 137: palmer.
Closest hotel in palmer is The Trainmaster's Inn.
------------
Retrieving Results for Index 145: nigde.
No result... skipping.
------------
Retrieving Results for Index 152: sussex.
No result... skipping.
------------
Retrieving Results for Index 184: carnarvon.
Closest hotel

In [8]:
# Display the hotel dataframe
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
15,zonguldak,TR,41.2500,31.8333,
20,laguna,US,38.4210,-121.4238,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
37,bethel,US,41.3712,-73.4140,Hampton Inn Danbury
50,ponta do sol,PT,32.6667,-17.1000,1905 Zinos Palace
98,yarmouth,US,41.7057,-70.2286,Isaiah Hall Bed & Breakfast
106,geraldton,AU,-28.7667,114.6000,Ocean Centre Hotel - Stay Beachside in Geraldton!
137,palmer,US,42.1584,-72.3287,The Trainmaster's Inn
145,nigde,TR,37.8333,34.7500,
152,sussex,US,41.1834,-74.6663,
184,carnarvon,AU,-24.8667,113.6333,Hospitality Carnarvon


In [9]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
20,laguna,US,38.4210,-121.4238,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
37,bethel,US,41.3712,-73.4140,Hampton Inn Danbury
50,ponta do sol,PT,32.6667,-17.1000,1905 Zinos Palace
98,yarmouth,US,41.7057,-70.2286,Isaiah Hall Bed & Breakfast
106,geraldton,AU,-28.7667,114.6000,Ocean Centre Hotel - Stay Beachside in Geraldton!
137,palmer,US,42.1584,-72.3287,The Trainmaster's Inn
184,carnarvon,AU,-24.8667,113.6333,Hospitality Carnarvon
222,lata,PT,40.1629,-8.3327,Arouce Hostel
226,ukiah,US,39.1502,-123.2078,Hampton Inn Ukiah
274,karonga,MW,-9.9333,33.9333,Golden Sand Beach Lodge


In [10]:


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))